In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.prompts import ChatPromptTemplate

request_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문 관련 요청을 분류하는 로봇이야.
            주문 관련 요청을 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'
            """
        ),
        ("human", "{input}"),

    ]
)

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

: 

In [6]:
from langchain_core.runnables import RunnablePassthrough

classify_request_chain = RunnablePassthrough.assign(request_type=request_type_prompt | model)

In [7]:
classify_request_chain.invoke({"input": "백설기 2개 주문할게요"})

{'input': '백설기 2개 주문할게요',
 'request_type': AIMessage(content='주문 요청', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 98, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2eb980a1-3263-4bf6-9563-519ddfdc00ce-0')}

In [10]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class OrderItemData(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    quantity: conint(gt=0) = Field(..., description="Quantity of the product")
    price: condecimal(max_digits=10, decimal_places=2) = Field(..., description="Price of the product")

class CreateOrderData(BaseModel):
    user_id: int = Field(..., description="ID of the user creating the order")
    items: List[OrderItemData] = Field(..., description="List of items to order")

class UpdateOrderStatusData(BaseModel):
    order_id: int = Field(..., description="ID of the order to update")
    new_status: str = Field(..., description="New status of the order")

create_order_parser = PydanticOutputParser(pydantic_object=CreateOrderData)
update_order_parser = PydanticOutputParser(pydantic_object=UpdateOrderStatusData)

In [ ]:
template = """
너는 각 주문 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.
주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'

상품 정보
{products}

user_id: {user_id}


user input: {input}
"""

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["uesr_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions(), update_order_parser.get_format_instructions()]},
)

In [11]:
from langchain_core.prompts import ChatPromptTemplate

messages = [
    ("system", 
     "너는 각 주문 요청 처리 함수에 필요한 인자를 추출하는 로봇이야."
     "주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'"
     ),
    ("ai", "{format_instructions}"),
    ("human", "상품 정보/n{products}/n/nuser_id: {user_id}/nuser input: {input}"),
]

prompt = ChatPromptTemplate.from_messages(
    messages,
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions(), update_order_parser.get_format_instructions()]}
)

TypeError: ChatPromptTemplate.from_messages() got an unexpected keyword argument 'partial_variables'

In [12]:
from langchain_core.prompts import AIMessagePromptTemplate, HumanMessagePromptTemplate

human_message_prompt = HumanMessagePromptTemplate.from_template("상품 정보/n{products}/n/nuser_id: {user_id}/nuser input: {input}")
ai_message_prompt = AIMessagePromptTemplate.from_template("{format_instructions}")

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "너는 각 주문 요청 처리 함수에 필요한 인자를 추출하는 로봇이야."
     "주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'"),
    (ai_message_prompt, [create_order_parser.get_format_instructions(), update_order_parser.get_format_instructions()]),
    (human_message_prompt, None),
])

TypeError: expected str, got list

In [34]:
template = """
너는 각 주문 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.
출력에 다른 내용은 포함시키지 마.

상품 정보
{products}

user_id: {user_id}


user input: {input}
"""

from langchain_core.prompts import PromptTemplate

extract_args_prompt = PromptTemplate(
    template=template,
    input_variables=["uesr_id", "input", "products"],
    partial_variables={"format_instructions": [create_order_parser.get_format_instructions(), update_order_parser.get_format_instructions()]},
)

In [35]:
extract_args_chain = extract_args_prompt | model
extract_args_chain

PromptTemplate(input_variables=['input', 'products', 'user_id'], partial_variables={'format_instructions': ['The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"OrderItemData": {"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "quantity": {"description": "Quantity of the product", "exclusiveMinimum": 0, "title": "Quantity", "type": "integer"}, "price": {"anyOf": [{"type": "number"}, {"type": "string"}], "description": "Price of the product", "title": "Price"}}, "required": ["product_name", "quantity", "price"], 

In [32]:
import json
import os

current_dir = os.getcwd()

# JSON 파일 경로
json_file_path = os.path.join(current_dir, 'products.json')

# JSON 파일을 읽어와서 파싱
with open(json_file_path, 'r', encoding='utf-8') as file:
    products = json.load(file)

# 상품 정보 출력
for product in products:
    print(f"상품: {product['product_name']}, 기본 판매 수량: {product['quantity']}개, 기본 판매 수량의 가격: {product['price']}원")

상품: 떡케익5호, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 54000원
상품: 무지개 백설기 케익, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 51500원
상품: 미니 백설기, 기본 판매 수량: 35개, 기본 판매 수량의 가격: 31500원
상품: 개별 모듬팩, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 13500원


In [36]:
response = extract_args_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

AIMessage(content='user_id: 1\nproduct_name: 개별 모듬팩\nquantity: 2', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 202, 'total_tokens': 224}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa8776b7-970a-47ae-bf21-a273fe909cf5-0')

In [37]:
create_order_parser(response)

TypeError: 'PydanticOutputParser' object is not callable

In [38]:
create_order_parser.invoke(response)

OutputParserException: Invalid json output: user_id: 1
product_name: 개별 모듬팩
quantity: 2

In [ ]:
create_order_chain = extract_args_chain | 

In [ ]:
def request_types_route(info):
    print("request_types_route 함수로 전달된 데이터 -> ", info)
    if "주문 요청" in info["request_type"].content.lower():
        return RunnableLambda()
    elif "주문 변경" in info["request_type"].content.lower():
        return RunnableLambda()
    else:
        return RunnableLambda()